In [7]:
%%capture
%pip install langchain langchain-core langchain-openai langgraph dotenv

In [1]:
from dotenv import load_dotenv


load_dotenv()

True

In [8]:
from langchain_openai import ChatOpenAI
from langchain_core.tools import tool
from langchain.agents import create_agent
from langgraph.checkpoint.memory import InMemorySaver

# 1. Definir ferramentas
@tool
def consultar_preco(produto: str) -> str:
    """Consulta o preço de um produto no estoque."""
    precos = {
        "notebook": "R$ 3.500,00",
        "mouse": "R$ 150,00",
        "teclado": "R$ 250,00"
    }
    return precos.get(produto.lower(), f"Produto '{produto}' não encontrado")

@tool
def verificar_estoque(produto: str) -> str:
    """Verifica a quantidade disponível em estoque."""
    estoque = {
        "notebook": 5,
        "mouse": 42,
        "teclado": 18
    }
    qtd = estoque.get(produto.lower())
    if qtd:
        return f"{qtd} unidades disponíveis"
    return f"Produto '{produto}' não cadastrado"

# 2. Criar o modelo
model = ChatOpenAI(model="gpt-4o", temperature=0)

# 3. (Opcional) Memória para persistir conversas
memory = InMemorySaver()

# 4. Criar o agente
agent = create_agent(
    model="openai:gpt-4o",  # formato "provider:model"
    tools=[consultar_preco, verificar_estoque],
    checkpointer=memory,
    system_prompt="Você é um assistente útil."  # novo parâmetro
)

In [9]:
config = {"configurable": {"thread_id": "sessao-1"}}

In [10]:
response = agent.invoke(
    {"messages": [("user", "Qual o preço do notebook e tem em estoque?")]},
    config=config
)

print(response["messages"][-1].content)

O preço do notebook é R$ 3.500,00 e há 5 unidades disponíveis em estoque.


In [11]:
response = agent.invoke(
    {"messages": [("user", "Qual o produto eu pesquisei")]},
    config=config
)

print(response["messages"][-1].content)

Você pesquisou sobre o "notebook".


In [14]:
state = agent.get_state(config)

print("=== ESTADO ATUAL ===")
for msg in state.values["messages"]:
    print(f"{msg.type}: {msg.content[:100]}...")

=== ESTADO ATUAL ===
human: Qual o preço do notebook e tem em estoque?...
ai: ...
tool: R$ 3.500,00...
tool: 5 unidades disponíveis...
ai: O preço do notebook é R$ 3.500,00 e há 5 unidades disponíveis em estoque....
human: Qual o produto eu pesquisei...
ai: Você pesquisou sobre o "notebook"....
